Code to finetune MedSam2


In [ ]:
import os
import shutil
from collections import OrderedDict
import zipfile
import json
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path
import numpy as np

!apt-get install python3.10
!pip install Pytorch==2.3.1
!pip install connected-components-3d
!pip install idc-index
!pip install ipywidgets
!pip install itk
!pip install simpleitk
!pip install wsidicom
!pip install pydicom
!pip install monai
!pip install hydra-core
!pip install tensorflow[cuda]
!pip install TensorRT[cuda]
import torch




ERROR:pydicom.pixels.utils:No module named 'pylibjpeg'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pydicom/pixels/utils.py", line 1274, in _passes_version_check
    module = importlib.import_module(package_name, "__version__")
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'pylibjpeg'
ERROR:pydicom.pixels.utils:No module named 'pylibjpeg'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pydicom/pixels/utils.py", line 1274, in _passes_version_check
    module = importlib.import_module(package_name, "__version__")
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, i

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
ERROR: Could not find a version that satisfies the requirement Pytorch==2.3.1 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for Pytorch==2.3.1
  Using cached tensorrt-10.4.0.tar.gz (16 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorrt-cu12-10.4.0.tar.gz (18 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.4.0-py2.py3-none-any.whl size=17554 sha256=25e1b67acf9c68508e274a1abb2fe7c731adfe18be66e19e7abaa76b4800b922
  Stored in directory: /root/.cache/pip/wheels/76/26/d6/92ecf38513ce11b54032131c94c117db6f1b35752d00728b29
  Created wheel for TensorRT: filename=tensorrt-10.4.0-py2.py3-none-any.whl size=16337 sha256=d5fb8305d1902aa10b9a848cf276e1d514db4c77ab980d6cbafdc1e59d06ff5

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "SamStuff")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/SamStuff

/content/drive/MyDrive/SamStuff


In [ ]:


!python MedSAM2/pre_CT_MR.py \
-img_path Dataset002_FullTraining/imagesTr \
-img_name_suffix _0000.nii.gz \
-gt_path Dataset002_FullTraining/labelsTr \
-gt_name_suffix .nii.gz \
-output_path "AuxData/imagesTr" \
-num_workers 4 \
-modality CT \
-anatomy cln \
-window_level 40 \
-window_width 400


ori \# files len(names)=1228
after sanity check \# files len(names)=1228
Preprocessing training data:   0% 0/40 [00:00<?, ?it/s]
Preprocessing training data:   2% 1/40 [00:00<00:25,  1.56it/s]
Preprocessing training data:   8% 3/40 [00:03<00:40,  1.10s/it]
Preprocessing training data:  10% 4/40 [00:03<00:28,  1.25it/s]
Preprocessing training data:  15% 6/40 [00:04<00:19,  1.71it/s]
Preprocessing training data:  18% 7/40 [00:04<00:17,  1.88it/s]
Preprocessing training data:  20% 8/40 [00:06<00:31,  1.02it/s]
Preprocessing training data:  22% 9/40 [00:08<00:33,  1.07s/it]
Preprocessing training data:  28% 11/40 [00:34<03:07,  6.48s/it]
Preprocessing training data:  30% 12/40 [00:35<02:27,  5.28s/it]
Preprocessing training data:  32% 13/40 [00:36<01:47,  4.00s/it]
Preprocessing training data:  35% 14/40 [00:37<01:24,  3.25s/it]
Preprocessing training data:  38% 15/40 [00:37<01:00,  2.42s/it]
Preprocessing training data:  40% 16/40 [00:37<00:42,  1.78s/it]
Preprocessing training data:  42%

In [ ]:
!python MedSAM2/npz_to_npy.py \
    -npz_dir AuxData/imagesTr/npz_test/CT_cln \
    -npy_dir AuxData/imagesTr/npy \
    -num_workers 4

Converting npz to npy: 100% 848/848 [44:05<00:00,  3.12s/it]


Initial Model Training no longer used

In [ ]:
!python MedSAM2/finetune_sam2_img.py \
    -i ./PrepData/imagesTr/npy \
    -task_name MedSAM2-Collon \
    -work_dir ./work_dir \
    -batch_size 8 \
    -pretrain_model_path ./MedSAM2/checkpoints/sam2_hiera_tiny.pt \
    -model_cfg sam2_hiera_t.yaml

2024-09-02 18:21:35.766346: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-02 18:21:35.784809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 18:21:35.806446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 18:21:35.812935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 18:21:35.829112: I tensorflow/core/platform/cpu_feature_guar

Resume traning command

In [ ]:
!python MedSAM2/finetune_sam2_img.py \
    -i ./PrepData/imagesTr/npy \
    -task_name MedSAM2-Collon \
    -work_dir ./work_dir \
    -batch_size 8 \
    -pretrain_model_path ./MedSAM2/checkpoints/sam2_hiera_tiny.pt \
    -model_cfg sam2_hiera_t.yaml \
    -resume ./work_dir/MedSAM2-Collon-20240903-1219/medsam_model_latest.pth

2024-09-04 16:51:36.579837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 16:51:36.597989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 16:51:36.619182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 16:51:36.625581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 16:51:36.641187: I tensorflow/core/platform/cpu_feature_guar

Infer image

In [ ]:
!python MedSAM2/infer_medsam2_flare22.py \
    -data_root PrepData/imagesTr/npz_train/CT_cln \
    -pred_save_dir segs/medsam2 \
    -sam2_checkpoint  MedSAM2/checkpoints/sam2_hiera_tiny.pt \
    -medsam2_checkpoint ./work_dir/MedSAM2-Collon-20240904-1653/medsam_model_latest.pth \
    -model_cfg sam2_hiera_t.yaml \
    -bbox_shift 5 \
    -num_workers 10 --visualize

100% 30/30 [04:01<00:00,  8.06s/it]


In [ ]:
!python MedSAM2/pre_CT_MR.py \
-img_path PrepData/new \
-img_name_suffix _0000.nii.gz \
-gt_path Dataset002_FullTraining/labelsTr \
-gt_name_suffix .nii.gz \
-output_path "AuxData/new" \
-num_workers 4 \
-modality CT \
-anatomy cln \
-window_level 40 \
-window_width 400


ori \# files len(names)=1228
after sanity check \# files len(names)=0
Preprocessing training data: : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Preprocessing training data: : 0it [00:00, ?it/s]
Preprocessing testing data: : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Preprocessing testing data: : 0it [00:00, ?it/s]


In [ ]:
!python MedSAM2/metrics/compute_metrics_flare22.py \
    -s segs/medsam2 \
    -g PrepData/imagesTr/npz_train/CT_cln \
    -csv_dir ./medsam2

100% 30/30 [00:14<00:00,  2.14it/s]
